# The Language of Failure

Resource: http://rahul.gopinath.org/post/2020/08/03/simple-ddset/

Also: https://neumorphism.io/#e0e0e0

_Brief abstract/introduction/motivation.  State what the chapter is about in 1-2 paragraphs._
_Then, have an introduction video:_

In [ ]:
from bookutils import YouTubeVideo
YouTubeVideo("w4u5gCgPlmg")

**Prerequisites**

* _Refer to earlier chapters as notebooks here, as here:_ [Earlier Chapter](Fuzzer.ipynb).

In [ ]:
import bookutils

In [ ]:
import DeltaDebugger

## A Failing Program

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    # postcondition
    assert '<' not in out and '>' not in out

    return out

In [ ]:
from ExpectError import ExpectError

In [ ]:
BAD_INPUT = '<foo>"bar</foo>'

In [ ]:
with ExpectError(AssertionError):
    remove_html_markup(BAD_INPUT)

## Grammars

In [ ]:
import string

In [ ]:
SIMPLE_HTML_GRAMMAR = {
    "<start>":
        ["<html>"],

    "<html>":
        ["<plain-text>", "<tagged-text>"],

    "<plain-text>":
        ["", "<plain-char><plain-text>"],

    "<plain-char>":
        ["<letter>", "<digit>", "<other>"],

    "<tagged-text>":
        ["<opening-tag><html><closing-tag>",
         "<combined-tag>",
         "<opening-tag>"],

    "<opening-tag>":
        ["<lt><id><gt>",
         "<lt><id><attrs><gt>"],

    "<closing-tag>":
        ["<lt>/<id><gt>"],

    "<combined-tag>":
        ["<lt><id><attrs>/<gt>"],

    "<lt>": [ "<" ],
    "<gt>": [ ">" ],

    "<attrs>":
        ["<attr>", "<attr> <attrs>" ],

    "<attr>":
        ["<id>='<plain-text>'",
         '<id>="<plain-text>"'],

    "<id>":
        ["<letter>", "<id><letter>", "<id><digit>"],

    "<letter>": list(string.ascii_letters),
    "<digit>": list(string.digits),
    "<other>": list(string.punctuation.replace('<', '').replace('>', ''))
}

## Fuzzing

In [ ]:
from fuzzingbook.GrammarFuzzer import GrammarFuzzer

In [ ]:
simple_html_fuzzer = GrammarFuzzer(SIMPLE_HTML_GRAMMAR)

In [ ]:
for i in range(20):
    fuzz_html = simple_html_fuzzer.fuzz()
    print(repr(fuzz_html))

## Derivation Trees

In [ ]:
from fuzzingbook.Parser import display_tree

In [ ]:
fuzz_html

In [ ]:
display_tree(simple_html_fuzzer.derivation_tree)

## Parsing

In [ ]:
from fuzzingbook.Parser import EarleyParser

In [ ]:
simple_html_parser = EarleyParser(SIMPLE_HTML_GRAMMAR)

In [ ]:
bad_input_tree = list(simple_html_parser.parse(BAD_INPUT))[0]

In [ ]:
display_tree(bad_input_tree)

In [ ]:
from fuzzingbook.GrammarFuzzer import tree_to_string, all_terminals

In [ ]:
tree_to_string(bad_input_tree)

## Mutating the Tree

In [ ]:
from fuzzingbook.Grammars import is_valid_grammar

In [ ]:
class TreeMutator:
    def __init__(self, grammar, tree, log=False):
        assert is_valid_grammar(grammar)
        self.grammar = grammar
        self.tree = tree
        self.log = log

In [ ]:
class TreeMutator(TreeMutator):
    def get_subtree(self, path, tree=None):
        if tree is None:
            tree = self.tree

        node, children = tree

        if not path:
            return tree

        return self.get_subtree(path[1:], children[path[0]])

In [ ]:
def bad_input_tree_mutator():
    return TreeMutator(SIMPLE_HTML_GRAMMAR, bad_input_tree, log=2)    

In [ ]:
bad_input_tree_mutator().get_subtree([0, 0, 1, 0])

In [ ]:
class TreeMutator(TreeMutator):
    def new_tree(self, start_symbol):
        if self.log >= 2:
            print(f"Creating new tree for {start_symbol}")

        fuzzer = GrammarFuzzer(self.grammar, start_symbol=start_symbol)
        fuzzer.fuzz()
        return fuzzer.derivation_tree

In [ ]:
bad_input_tree_mutator().new_tree('<plain-text>')

In [ ]:
class TreeMutator(TreeMutator):
    def mutate(self, path, tree=None):
        if tree is None:
            tree = self.tree

        node, children = tree

        if not path:
            return self.new_tree(node)

        head = path[0]
        new_children = (children[:head] +
                        [self.mutate(path[1:], children[head])] +
                        children[head + 1:])
        return node, new_children

In [ ]:
bad_input_tree_mutator().mutate([0])

In [ ]:
tree_to_string(bad_input_tree_mutator().mutate([0, 0, 1, 0]))

## Generalizing Trees

In [ ]:
class TreeGeneralizer(TreeMutator):
    def __init__(self, grammar, tree, test,
                 max_tries_for_abstraction=10,
                 **kwargs):
        super().__init__(grammar, tree, **kwargs)
        self.test = test
        self.max_tries_for_abstraction = max_tries_for_abstraction

In [ ]:
class TreeGeneralizer(TreeGeneralizer):
    def test_tree(self, tree):
        s = tree_to_string(tree)
        if self.log:
            print(f"Testing {repr(s)}...", end="")
        try:
            self.test(s)
        except Exception as exc:
            if self.log:
                print(f"FAIL ({type(exc).__name__})")
            ret = False
        else:
            if self.log:
                print(f"PASS")
            ret = True

        return ret

In [ ]:
class TreeGeneralizer(TreeGeneralizer):
    def can_generalize(self, path, tree=None):
        for i in range(self.max_tries_for_abstraction):
            mutated_tree = self.mutate(path, tree)
            if self.test_tree(mutated_tree):
                # Failure no longer occurs; cannot abstract
                return False
            
        return True

In [ ]:
def bad_input_tree_generalizer():
    return TreeGeneralizer(SIMPLE_HTML_GRAMMAR, bad_input_tree,
                           remove_html_markup, log=True)    

In [ ]:
bad_input_tree_generalizer().can_generalize([0])

In [ ]:
bad_input_tree_generalizer().can_generalize([0, 0, 1, 0])

In [ ]:
bad_input_tree_generalizer().can_generalize([0, 0, 0])

In [ ]:
class TreeGeneralizer(TreeGeneralizer):
    def find_paths(self, predicate, path=None, tree=None):
        if path is None:
            path = []
        if tree is None:
            tree = self.tree
            
        node, children = self.get_subtree(path)

        if predicate(path, tree):
            if self.log:
                node, children = self.get_subtree(path)
            return [path]

        paths = []
        for i, child in enumerate(children):
            child_node, _ = child
            if child_node in self.grammar:
                paths += self.find_paths(predicate, path + [i])

        return paths        
    
    def generalizable_paths(self):
        return self.find_paths(self.can_generalize)

In [ ]:
bad_input_tree_generalizer().generalizable_paths()

In [ ]:
class TreeGeneralizer(TreeGeneralizer):
    def generalize_path(self, path, tree=None):
        if tree is None:
            tree = self.tree

        node, children = tree

        if not path:
            return node, []

        head = path[0]
        new_children = (children[:head] +
                        [self.generalize_path(path[1:], children[head])] +
                        children[head + 1:])
        return node, new_children

In [ ]:
all_terminals(bad_input_tree_generalizer().generalize_path([0, 0, 0]))

In [ ]:
class TreeGeneralizer(TreeGeneralizer):
    def generalize(self):
        tree = self.tree
        for path in self.generalizable_paths():
            tree = self.generalize_path(path, tree)
            
        return tree

In [ ]:
all_terminals(bad_input_tree_generalizer().generalize())

## Putting it all Together

In [ ]:
from DeltaDebugger import CallCollector, is_reducible

In [ ]:
import copy

In [ ]:
class DDSetDebugger(CallCollector):
    def __init__(self, grammar, **kwargs):
        super().__init__(**kwargs)
        self.grammar = grammar
        self.parser = EarleyParser(grammar)

    def generalized_args(self, **kwargs):
        generalized_args = copy.deepcopy(self.args())

        for arg in self.args():
            def test(value):
                return self.call({arg: value})

            value = self.args()[arg]
            if is_reducible(value):
                tree = list(self.parser.parse(value))[0]
                tt = TreeGeneralizer(self.grammar, tree, test, **kwargs)
                generalized_args[arg] = all_terminals(tt.generalize())

        return generalized_args

In [ ]:
with DDSetDebugger(SIMPLE_HTML_GRAMMAR) as dd:
    remove_html_markup(BAD_INPUT)

In [ ]:
dd.generalized_args()

# Synopsis

<!-- Automatically generated. Do not edit. -->



_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

You can use `int_fuzzer()` as:

```python
print(int_fuzzer())
```
```python
=> 76.5

```


## _Section 1_

\todo{Add}

## _Section 2_

\todo{Add}

### Excursion: All the Details

This text will only show up on demand (HTML) or not at all (PDF). This is useful for longer implementations, or repetitive, or specialized parts.

### End of Excursion

## _Section 3_

\todo{Add}

_If you want to introduce code, it is helpful to state the most important functions, as in:_

* `random.randrange(start, end)` - return a random number [`start`, `end`]
* `range(start, end)` - create a list with integers from `start` to `end`.  Typically used in iterations.
* `for elem in list: body` executes `body` in a loop with `elem` taking each value from `list`.
* `for i in range(start, end): body` executes `body` in a loop with `i` from `start` to `end` - 1.
* `chr(n)` - return a character with ASCII code `n`

In [ ]:
import random

In [ ]:
def int_fuzzer():
    """A simple function that returns a random integer"""
    return random.randrange(1, 100) + 0.5

In [ ]:
# More code
pass

## _Section 4_

\todo{Add}

## Synopsis

_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

You can use `int_fuzzer()` as:

In [ ]:
print(int_fuzzer())

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_

## Reducing Trees

In [ ]:
from DeltaDebugger import DeltaDebugger

In [ ]:
import copy

In [ ]:
from IPython.display import display

In [ ]:
class TreeHDDReducer(TreeGeneralizer):
    def _reduce(self, path, tree):
        """This is HDD"""

        node, children = self.get_subtree(path, tree)
            
        if len(path) >= 1:
            parent, parent_children = self.get_subtree(path[:-1], tree)
 
            assert parent_children[path[-1]] == (node, children)

            def test_children(children):
                parent_children[path[-1]] = (node, children)
                s = tree_to_string(tree)
                self.test(s)

            with DeltaDebugger() as dd:
                test_children(children)
            
            # display(display_tree(tree))

            children = dd.min_args()['children']
            parent_children[path[-1]] = (node, children)
        
        for i, child in enumerate(children):
            self._reduce(path + [i], tree)
            
        return tree

    def reduce(self):
        return self._reduce([], self.tree)

In [ ]:
def bad_input_tree_hdd_reducer():
    return TreeHDDReducer(SIMPLE_HTML_GRAMMAR, copy.deepcopy(bad_input_tree),
                       remove_html_markup, log=True)    

In [ ]:
all_terminals(bad_input_tree_hdd_reducer().reduce())

## More Reducing Trees

In [ ]:
class TreeReducer(TreeGeneralizer):
    def new_min_tree(self, start_symbol):
        if self.log >= 2:
            print(f"Creating new minimal tree for {start_symbol}")

        fuzzer = GrammarFuzzer(self.grammar, start_symbol=start_symbol,
                               min_nonterminals=0,
                               max_nonterminals=0)
        fuzzer.fuzz()
        return fuzzer.derivation_tree

In [ ]:
def bad_input_tree_reducer():
    return TreeReducer(SIMPLE_HTML_GRAMMAR, bad_input_tree,
                       remove_html_markup, log=2)    

In [ ]:
tree_to_string(bad_input_tree_reducer().new_min_tree('<start>'))

In [ ]:
class TreeReducer(TreeReducer):
    def reduce_path(self, path, tree=None):
        if tree is None:
            tree = self.tree

        node, children = tree

        if not path:
            return self.new_min_tree(node)

        head = path[0]
        new_children = (children[:head] +
                        [self.reduce_path(path[1:], children[head])] +
                        children[head + 1:])
        return node, new_children

In [ ]:
tree_to_string(bad_input_tree_reducer().reduce_path([0, 0, 1, 0]))

In [ ]:
class TreeReducer(TreeReducer):
    def can_reduce(self, path, tree=None):
        reduced_tree = self.reduce_path(path, tree)
        if self.test_tree(reduced_tree):
            # Failure no longer occurs; cannot reduce
            return False

        return True

In [ ]:
class TreeReducer(TreeReducer):
    def reducible_paths(self):
        return self.find_paths(self.can_reduce)

In [ ]:
bad_input_tree_reducer().reducible_paths()

In [ ]:
class TreeReducer(TreeReducer):
    def reduce(self):
        tree = self.tree
        for path in self.reducible_paths():
            tree = self.reduce_path(path, tree)
            
        return tree

In [ ]:
all_terminals(bad_input_tree_reducer().reduce())